# PRIMEIRO PASSO: Criar a tabela no banco a partir do CSV

#### Aqui estamos lendo o CSV e criando uma tabela chamada "dados_limpos"
#### no banco SQLite. Se a tabela já existir, ela será substituída.

In [ ]:
import pandas as pd
import sqlite3

df = pd.read_csv('data/silver/dados_limpos.csv')

conn = sqlite3.connect('data/pipeline.db')

df.to_sql('dados_limpos', conn, if_exists='replace', index=False)

print("Tabelas existentes no banco:")
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn))


Tabelas existentes no banco:
           name
0   dim_country
1  dim_customer
2      dim_date
3   dim_product
4          fact
5  dados_limpos



# QUERY 1: Quantos registros temos no banco

#### Aqui queremos apenas ter uma visão geral: quantas linhas existem no banco.
#### Isso ajuda a entender o tamanho da base de dados antes de fazer qualquer análise.

In [18]:
query = """
SELECT COUNT(*) AS total_registros
FROM dados_limpos
"""
resultado = pd.read_sql_query(query, conn)
print("\nTotal de registros:", resultado['total_registros'].values[0])


Total de registros: 541909


# QUERY 2: Top 10 produtos que mais vendem

#### Vamos descobrir quais produtos geraram mais receita.
#### Para isso:
#### - Agrupamos por Description (nome do produto)
#### - Somamos a quantidade vendida e calculamos a receita (Quantity * UnitPrice)
#### - Ordenamos do maior para o menor e pegamos os 10 primeiros

In [27]:
query = """
SELECT
    Description AS produto,
    SUM(Quantity) AS quantidade_total_vendida,
    ROUND(SUM(Quantity * UnitPrice), 2) AS receita_total
FROM dados_limpos
WHERE Description IS NOT NULL
GROUP BY Description
ORDER BY receita_total DESC
LIMIT 10
"""
print("\nTop 10 Produtos Mais Vendidos:")
print(pd.read_sql_query(query, conn))


Top 10 Produtos Mais Vendidos:
                              produto  quantidade_total_vendida  receita_total
0                      DOTCOM POSTAGE                      1707      206245.48
1            REGENCY CAKESTAND 3 TIER                     12980      164762.19
2  WHITE HANGING HEART T-LIGHT HOLDER                     39122       99846.98
3                       PARTY BUNTING                     18022       98302.98
4             JUMBO BAG RED RETROSPOT                     47363       92356.03
5                             mailout                     44082       83786.10
6                  RABBIT NIGHT LIGHT                     30646       66756.59
7                             POSTAGE                      6353       66230.64
8     PAPER CHAIN KIT 50'S CHRISTMAS                      18902       63791.94
9       ASSORTED COLOUR BIRD ORNAMENT                     36221       58959.73


# QUERY 3: Vendas por dia

#### Aqui queremos ver como as vendas variam ao longo do tempo.
#### - Transformamos InvoiceDate em data (sem horário)
#### - Contamos o número de vendas distintas por dia
#### - Calculamos a receita total do dia
#### - Ordenamos pela data para ver a evolução cronológica

In [28]:
query = """
SELECT
    DATE(InvoiceDate) AS data,
    COUNT(DISTINCT InvoiceNo) AS total_vendas,
    ROUND(SUM(Quantity * UnitPrice), 2) AS receita_total
FROM dados_limpos
GROUP BY DATE(InvoiceDate)
ORDER BY data
"""
print("\nVendas por Dia:")
print(pd.read_sql_query(query, conn).head())


Vendas por Dia:
   data  total_vendas  receita_total
0  None         25900     9747747.93


# QUERY 4: Clientes por país

#### Agora queremos ver a distribuição dos clientes por país.
#### - Contamos os clientes únicos por país
#### - Calculamos o ticket médio (quanto, em média, cada cliente gastou)
#### - Excluímos registros sem CustomerID, porque não podemos contar clientes inexistentes
#### - Ordenamos pelos países com mais clientes

In [29]:
query = """
SELECT
    Country AS pais,
    COUNT(DISTINCT CustomerID) AS total_clientes,
    ROUND(AVG(Quantity * UnitPrice), 2) AS ticket_medio
FROM dados_limpos
WHERE CustomerID IS NOT NULL
GROUP BY Country
ORDER BY total_clientes DESC
"""
print("\nClientes por País:")
print(pd.read_sql_query(query, conn))



Clientes por País:
                    pais  total_clientes  ticket_medio
0         United Kingdom            3950         18.70
1                Germany              95         23.35
2                 France              87         23.17
3                  Spain              31         21.62
4                Belgium              25         19.77
5            Switzerland              21         29.70
6               Portugal              19         19.64
7                  Italy              15         21.03
8                Finland              12         32.12
9                Austria              11         25.32
10                Norway              10         32.38
11           Netherlands               9        120.06
12               Denmark               9         48.25
13       Channel Islands               9         26.50
14             Australia               9        108.88
15                Sweden               8         79.21
16                 Japan               8     

#### Fechamos a conexão com o banco só no final, depois que todas as consultas foram feitas.

In [ ]:

conn.close()
print("\nConexão encerrada")


Conexão encerrada com sucesso ✅
